In [1]:
# Importing libraries
from bs4 import BeautifulSoup
import requests
import time
import re
import numpy as np
from time import sleep
from random import randint
import pymongo
from pymongo import MongoClient
import json
import http.client, urllib.parse

In [2]:
#Function to save html files
def saveString(html, filename):
	try:
		file = open(filename,"w")
		file.write(str(html))
		file.close()
	except Exception as ex:
		print('Error: ' + str(ex))

#Function to load html files
def loadString(f):
	try:
		html = open(f, "r", encoding='utf-8').read()
		return(html)
	except Exception as ex:
		print('Error: ' + str(ex))

#To replicate actual user behaviour on a browser
header_vars= {
		"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
		"accept-encoding": "gzip, deflate, br",
		"accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
		"cache-control": "max-age=0",
		"cookie": "__uzma=656e04b8-689b-4af5-956e-ecc0e52d0459; __uzmb=1642384927; __uzmc=508531043362; __uzmd=1642384927; __uzme=6174; __uzmf=7f3000f54b972e-7a1f-4f3e-8597-c9b82815fc4191fc0b70a70e438410; __ssds=2; __ssuzjsr2=a9be0cd8e; __uzmaj2=3071bfbf-a34d-454c-a9b3-ce974951c0bb; __uzmbj2=1642384928; __uzmcj2=580381076551; __uzmdj2=1642384928; ak_bmsc=18A13C834C6EE2297609BC196B251B93~000000000000000000000000000000~YAAQzXpCF5+0heN9AQAAgTJcag73FiKCg4/zBZQHKJ+UoBBL+UD4HclUDuPWnRmkeXOdTx7A0Rrgy+3RzouXPgdTiNrAgTfzwNECXPOYfT0aza4l8dBHR2ym+XRTO+wWXJZwspKHBSt4ikMFUSogWEHrRfyhD9KnjQkrs66uP3QwL3KIaGK2melsvb3h+2/VfD57hJsjeFRsilniQ8XLQGixWON2epSxOI5EJURojzF3VXNqZr79lSq+NicYSDY8aU08HjC3YtotIiJ8eguuZv5HxjDPFbs1BbBpUYodxKqisLbu1eEYPCgYqARd6Ry13MBzIJjtDhMR69JJK6dLXC4c1weH+sBsPvaqA9IhR9yXVVePguSlrcluE5uG/eEbl1aVe85fLw==; s=CgAD4ACBh51A0NjVjNTIxZWUxN2UwYWI4Y2JiMDRmMjU4ZmZmNmU4ZDmQ+mTx; npii=btguid/65c521ee17e0ab8cbb04f258fff6e8d965a865fb^cguid/65c5266817e0ab8cc1224a83ff6a881965a865fb^; ebay=%5Esbf%3D%23000000%5E; dp1=bu1p/QEBfX0BAX19AQA**65a865fb^bl/US65a865fb^; nonsession=BAQAAAX42/BiUAAaAADMABWPHMns5NDEyMgDKACBlqGX7NjVjNTIxZWUxN2UwYWI4Y2JiMDRmMjU4ZmZmNmU4ZDkAywACYeYGAzE0itmBEsw1q38j8xYAykLMK73No8c*; bm_sv=0601DA57A7CA54406136CDCACF44DF42~MSyqjSDnWVhHfr3JjTO7k1sCODPgV4YV2GZjV2GAGLd4fUNZybtHajYnmslko8iQ/26HcsXohlAlwP850fnv4hTSsYycvRBVE/jmVQV4/aTXjODnySUtVq/5Etyuz8mqk1Ww6qR0s8BC8V5YKLDSaw==",
		"sec-ch-ua-mobile": "?0",
		"sec-ch-ua-platform": "macOS",
		"sec-fetch-dest": "document",
		"sec-fetch-mode": "navigate",
		"sec-fetch-site": "none",
		"sec-fetch-user": "?1",
		"upgrade-insecure-requests": "1",
		"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36"
		}

**(1)  Yelp uses GET requests for its search. Write a program that searches on yelp.com for the top-40 “Donut Shop” in the San Francisco area. Save each search result page to disk, “sf_donut_shop_search_page_[PN].htm”**

In [3]:
URL = "https://www.yelp.com/search?find_desc=Donut+Shop&find_loc=San+Francisco%2C+CA&start="

#Create an empty list to store urls
list = []
pages = ["0", "10", "20", "30"]


for pageNum in pages:
	page = requests.get(URL + pageNum, headers = header_vars)
    
    # Create a beautifulsoup object
	soup = BeautifulSoup(page.content, "html.parser")
    
    #Downloading each page in .htm format
	saveString(soup, "sf_donut_shop_search_page_"+str(pages.index(pageNum)+1)+".htm")
    
    #Wait after downloading each page
	sleep(5)

**(2) Write new code that opens the search result pages saved in (1) and parses out all shop information (search rank, name, linked URL [this store’s Yelp URL], star rating, number of reviews, store tags, “$” signs, delivery / dine-in tags, and whether you can order through Yelp). Skip all “Sponsored Results”.**

**(3) Adjust code in (2) to create a MongoDB collection called “sf_donut_shops” that stores all the extracted shop information, one document for each shop.**

In [6]:
#Connect to MongoDB client
client = MongoClient('localhost', 27017)
db = client['sf_donut_shops']
col = db['sf_donut_shops']

#Iterating through all '.htm' pages to retrieve information from each page
for pageNum in pages:
    #Download page in htm format
	htmlFile = loadString("sf_donut_shop_search_page_"+str(pages.index(pageNum)+1)+".htm")
	soup2 = BeautifulSoup(htmlFile, 'html.parser')
	tags = soup2.select('div[class*="hoverable"]')
    
    #Run a loop to print all item details on the page
	for i in range(2, 12):
		tag = tags[i]

        ### Search Rank
		rank = tag.find('h3').find('span').get_text(separator=" ").split(".")[0]      
    
        ### Shop Yelp URL
		link = "https://www.yelp.com" + tag.find('a').get('href')
		name = tag.find('h3').find('span').find('a').text
		regex1 = re.compile('.*reviewCount.*')
		reviewCount = tag.find("span", {"class" : regex1}).get_text()
    
		regex2 = re.compile('.*i-stars.*')
		rating = tag.find("div", {"class" : regex2})['aria-label']
    
		regex3 = re.compile('.*priceRange.*')

        ### Order through Yelp
		orderThroughYelp = False
		if tag.select('a[class*="platformSearchAction"]'):
			orderThroughYelp = True
    
        ### Dollars for restaurant
		try:
			dollars = tag.find("span", {"class" : regex3}).text
		except:
			dollars = ""        


        ### Store Tags
		buttons = tag.find_all('button')
		storeTagList = []    
		for i in range(len(buttons)):
			storeTags = buttons[i].text
			storeTagList.append(storeTags)   
    
        ### Delivery Tags
		delTagList = []
		if tag.find("div", {"data-testid": "TRUSTED_PROPERTY"}):
			mydivs = tag.find("div", {"data-testid": "TRUSTED_PROPERTY"})
			delTags = mydivs.find_all('p')    
			for i in range(len(delTags)):
				deliveryTags = delTags[i].find('span').text
				delTagList.append(deliveryTags)       

        ### Delivery Tag Values
		delTagVal = []
		if tag.find("div", {"data-testid": "TRUSTED_PROPERTY"}):
			mydivs = tag.find("div", {"data-testid": "TRUSTED_PROPERTY"})
			valTags = mydivs.find_all("span", {"role": "img"})
			for i in range(len(valTags)):
				if mydivs.select('span[class*="close-v2"]'):
					delTagVal.append(0)
				elif mydivs.select('span[class*="checkmark"]'):
					delTagVal.append(1)
    

		delTagDict = dict(zip(delTagList, delTagVal))
        
        #Printing shop details from Yelp on console
		print("Shop Rank: ", rank)
		print("Shop Name: ", name)
		print("Shop URL: ", link)
		print("Rating: ", rating)
		print("Number of Reviews: ", reviewCount)
		print("Store Tags: ", storeTagList)
		print("Price Range: ", dollars)
		print("Delivery Tags: ", delTagDict)
		print("If you can order through Yelp: ", orderThroughYelp)
		print("\n")
        
        #Creating an array for keys (which will be part of a dictionary)
		varList = ["searchRank","productName", "productLink", "reviews", "ratingNum", "priceRange", "yelpOrder", "StoreTags", "deliveryTags"]

        #Creating an array for values of the above keys
		paramList = [int(rank), name, link, reviewCount, rating, dollars, orderThroughYelp, storeTagList, delTagDict]
        
        #Creating  a dictionary of shop details
		sequence = dict(zip(varList, paramList))
         
        #Inserting each row in MongoDB
		col.insert_one(sequence)
# 		col.drop()

                        

IndexError: list index out of range

### sf_donut_shops collection

In [ ]:
#Connect to MongoDB client
client = MongoClient('localhost', 27017)
db = client['sf_donut_shops']
col = db['sf_donut_shops']

query = {}

cursor = col.find(query)
try:
	for doc in cursor:
		print(doc)
finally:
	client.close()

**(4) Write a _new_ piece of code that reads all URLs stored in “sf_donut_shops” and download each shop page.  Store the page to disk, “sf_donut_shop_[SR].htm”**

In [ ]:
#Connect to MongoDB client
client = MongoClient('localhost', 27017)
db = client['sf_donut_shops']
col = db['sf_donut_shops']

query = {}

projection = {}
projection["productLink"] = 1.0
projection["_id"] = 0.0

cursor = col.find(query, projection = projection)

#Create an empty product list
prodLinkList = []

try:   
	for doc in cursor:
        #Append shop page urls to product list
		prodLinkList.append(doc.get('productLink'))
finally:
	client.close()

#Iterating through all urls and downloading shop pages in '.htm' format
for prodURL in prodLinkList:
	prodPage = requests.get(prodURL, headers = header_vars)
	soup3 = BeautifulSoup(prodPage.content, "html.parser")
	saveString(soup3, "sf_donut_shop_"+str(prodLinkList.index(prodURL)+1)+".htm")
	sleep(5)

**(5) Write new code that reads the 40 shop pages saved in (4) and parses each shop’s address, phone number, and website.**

**(6) Sign up for a free account with https://positionstack.com/  Adjust your code in (5) to query each shop address’ geolocation (long, lat). Update each shop document on the MongoDB collection “sf_donut_shops” to contain the shop’s address, phone number, website, and geolocation. Lastly, place an index on the shop’s search rank.**

In [ ]:
shopInfo = {}
client = MongoClient('localhost', 27017)
db = client['sf_donut_shops']
col = db['sf_donut_shops']

#Iterating through downloaded '.htm' files and retrieving shop details
for prodURL in prodLinkList:
	rank = prodLinkList.index(prodURL)+1
	prodHtmFile = loadString("sf_donut_shop_"+str(prodLinkList.index(prodURL)+1)+".htm")
	soup4 = BeautifulSoup(prodHtmFile, 'html.parser')
	regex4 = re.compile('css-1vhakgw border--top.*')
	sections = soup4.find_all("div", {"class" : regex4})
	whole_text=[i.text for i in sections]
    
    #Retrieve business website
	try:
		website_id=[('Business website' in i) for i in whole_text].index(True)
		website=whole_text[website_id].replace("Business website","")
	except:
		website=''
    
    #Update website details in MongoDB collection
	col.update_one({"searchRank":rank}, {"$set":{"Shop Website":website}})
    
    #Retrieve Shop Phone Number
	try:
		phone_number_id=[('Phone number' in i) for i in whole_text].index(True)
		phone_number=whole_text[phone_number_id].replace("Phone number","")
	except:
		phone_number=''

    #Update phone number details in MongoDB collection
	col.update_one({"searchRank":rank}, {"$set":{"Shop Phone":phone_number}})

    #Retrieve Shop Address
	try:
		address_id = [('Get Directions' in i) for i in whole_text].index(True)
		address = whole_text[address_id].replace("Get Directions","")
	except:
		address =''

    #Update address details in MongoDB collection
	col.update_one({"searchRank":rank}, {"$set":{"Shop Address":address}})
        
	apiURL = "http://api.positionstack.com/v1/forward?access_key=c18a3f5e39fab26597545ffdcf4e8291&query=" + address
	r= requests.get(apiURL)
	data = r.json()
	latitude = data["data"][0]['latitude']
	longitude = data["data"][0]['longitude']
	geoLocation = [latitude, longitude]

    #Update search rank in MongoDB collection
	col.update_one({"searchRank":rank}, {"$set":{"Geo Location":geoLocation}})

    #Print shop details available on Donut shop page in console
	print("Shop: ", rank)
	print("Shop Website: ", website)
	print("Shop Phone: ", phone_number)
	print("Shop Address: ", address)
	print("Geo Location: ", geoLocation)
	print("\n")
    
##Adding index on search rank
col.create_index('searchRank')

#Close MongoDB client
client.close()
    

### Display final collection

In [ ]:
#Connect to MongoDB client
client = MongoClient('localhost', 27017)
db = client['sf_donut_shops']
col = db['sf_donut_shops']

col.create_index('searchRank')
print(col.index_information())

query = {}

cursor = col.find(query)
try:
	for doc in cursor:
		print(doc)
finally:
	client.close()